# Pré-processamento do Dataset da Olist

Origem do dataset: 
kaggle/olistbr-brazilian-ecommerce

Segundo a descrição do kaggle, este dataset contém:

dados coletados entre 2016 e 2018;
~100 mil pedidos de múltiplos marketplaces no Brasil.

## Para baixar os dados
Entre no link do Kaggle e faça download dos datasets e coloque-os na pasta chamada data:

- olist_customers_dataset
- olist_orders_dataset
- olist_order_items_dataset
- olist_geolocation_dataset
- olist_products_dataset.csv

In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
cliente_df = pd.read_csv("data/olist_customers_dataset.csv")
pedido_df = pd.read_csv("data/olist_orders_dataset.csv")
pedido_itens_df = pd.read_csv("data/olist_order_items_dataset.csv")
produto_df = pd.read_csv("data/olist_products_dataset.csv")
geo_df = pd.read_csv("data/olist_geolocation_dataset.csv")

In [3]:
geo_df.columns = [col.replace("geolocation_", "") for col in geo_df.columns]

In [4]:
datasets_preview = {
    "cliente": cliente_df.head(n=2),
    "pedido": pedido_df.head(n=2),
    "pedido_itens": pedido_itens_df.head(n=2),
    "produto": produto_df.head(n=2),
    "geo": geo_df.head(n=2)
}

In [5]:
for name, dataset in datasets_preview.items():
    print(f"\t\t\t\t**{name}**")
    display(dataset)

				**cliente**


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP


				**pedido**


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00


				**pedido_itens**


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.9,19.93


				**produto**


,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0


				**geo**


,zip_code_prefix,lat,lng,city,state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP


## Preparando o dataset de pedidos


**Passo 1**

- incluir as colunas customer_zip_code_prefix, customer_city e customer_state a partir do dataframe cliente_df
- incluir a coluna product_category_name a partir do dataframe produto_df
- remover colunas customer_id


In [6]:
df = cliente_df \
    .merge(cliente_df.drop("customer_unique_id", 1), on="customer_id") \
    .merge(pedido_df, on="customer_id") \
    .merge(pedido_itens_df, on="order_id") \
    .merge(produto_df, on="product_id")     

In [7]:
df = df.drop(["customer_zip_code_prefix_x", "customer_city_x", "customer_state_x"], axis=1)

In [8]:
df.columns = [col.replace("_y", "") for col in df.columns]

**Passo 2**

incluir colunas de geolocalização (latitude, longitude)

In [9]:
geo_df["lat_long"] = geo_df.apply(lambda row: (row["lat"], row["lng"]), axis=1)

In [10]:
lat_long_dict = geo_df.set_index("zip_code_prefix")["lat_long"].to_dict()

In [12]:
for user_type in ["customer"]:
    zip_codes = df[f"{user_type}_zip_code_prefix"].tolist()
    lat_long_list = []
    for zc in tqdm(zip_codes):
        lat_long_list.append(lat_long_dict.get(zc, (0, 0)))
    lats, longs = list(zip(*lat_long_list))
    df[f"{user_type}_lat"] = lats
    df[f"{user_type}_long"] = longs

100%|██████████| 112650/112650 [00:00<00:00, 445513.82it/s]


In [13]:
df.columns

Index(['customer_id', 'customer_unique_id', 'customer_zip_code_prefix',
       'customer_city', 'customer_state', 'order_id', 'order_status',
       'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'order_item_id', 'product_id',
       'seller_id', 'shipping_limit_date', 'price', 'freight_value',
       'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'customer_lat', 'customer_long'],
      dtype='object')

In [14]:
df = df.drop(['customer_unique_id', 'customer_zip_code_prefix', 'order_status',
       'order_purchase_timestamp', 'order_approved_at','order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date', 'seller_id', 
       'shipping_limit_date','freight_value','product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm'], axis=1)

In [15]:
df.columns

Index(['customer_id', 'customer_city', 'customer_state', 'order_id',
       'order_item_id', 'product_id', 'price', 'product_category_name',
       'customer_lat', 'customer_long'],
      dtype='object')

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112650 entries, 0 to 112649
Data columns (total 10 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   customer_id            112650 non-null  object 
 1   customer_city          112650 non-null  object 
 2   customer_state         112650 non-null  object 
 3   order_id               112650 non-null  object 
 4   order_item_id          112650 non-null  int64  
 5   product_id             112650 non-null  object 
 6   price                  112650 non-null  float64
 7   product_category_name  111047 non-null  object 
 8   customer_lat           112650 non-null  float64
 9   customer_long          112650 non-null  float64
dtypes: float64(3), int64(1), object(6)
memory usage: 9.5+ MB


In [17]:
df.to_csv("olist_processado.csv", index=False)